# Marketing .pptx slide deck automation

This notebook reads related data from various sources and generate PowerPoint slide deck based on fully paramterized PowerPoint template.

In [ ]:
import pandas as pd
import numpy as np


##### pptx automation
from AMM_Automation_Program_PPT_Functions_QSR import *
from pptx import Presentation
from pptx.enum.dml import MSO_THEME_COLOR
from dateutil.parser import parse
import os, glob

import importlib, sys
importlib.reload(sys.modules['AMM_Automation_Program_PPT_Functions_QSR'])

In [ ]:
# loading input data

path = 'data\\'
xlsx_files = glob.glob(os.path.join(path,"*.xlsx"))

frames=[]

#for f in xlsx_files:
 #   market = f.split('.')[0].split('_')[1]
  #  df = pd.read_excel(f)
    
df_sheet_map = pd.read_excel(xlsx_files[0], sheet_name=None)

for locale in list(df_sheet_map.keys()):
    if len(locale) <= 3:
        df = df_sheet_map[locale]
        df.columns.values[3] = 'metric_name'
        df['locale'] = locale
        frames.append(df)
    else: # sample size sheet
        df_size = df_sheet_map[locale]
        df_size = df_size[['Locale','Final N']]
        df_size = df_size.set_index('Locale')
        locale_size_dict = df_size.to_dict()

df_all = pd.concat(frames)


#### GWI data for 1) Amazon Visitors 2) Streamming Users
df_amm = pd.read_excel('data/GWI/GWI data.xlsx')
df_amm['Streaming Users'] = df_amm['Amazon Prime Video Users'] + df_amm['Twitch Users']

#### Spend Data from PAID dashboard
spend_q42021 = pd.read_csv('data/Amazon Internal/Spend Q4 2021.csv')
spend_baseline = pd.read_csv('data/Amazon Internal/Spend Baseline.csv')

df_spend = pd.concat([spend_q42021, spend_baseline])

#### marketplace_id to country code mapping
country_map = {1:'US',3:'UK',4:'DE',7:'CA',35691:'IT',44551:'ES'}
df_spend['Locale'] = df_spend['marketplace_id'].map(country_map) 

#### merge with GWI data
df_amm = df_amm.merge(df_spend, how='left', left_on=['Wave','Locale'], right_on = ['wave','Locale'])
df_amm = df_amm[['Locale','Wave','Amazon Visitors','Streaming Users','spend']]

In [ ]:


def AMM_Metric(df):
    
    Q42021 = df[df['Wave']=='Q4 2021']
    Baseline = df[df['Wave']=='Baseline']
    
    # Amazon User %: Compare Amazon User in Q4 2021 vs. average of Q1-Q3 2021
    AMZN_Visitor_Pct =\
    (Q42021['Amazon Visitors'].values - Baseline['Amazon Visitors'].values)*100 / Baseline['Amazon Visitors'].values
    
    AMZN_Visitor_Pct = round(AMZN_Visitor_Pct[0],1).astype(str) + '%'
    
    
    # Streaming User: Using Prime Video User + Twitch User, Q4 2021 vs. Avg. of Q1-Q3 2021
    AMZN_Streaming_Pct =\
    (Q42021['Streaming Users'].values - Baseline['Streaming Users'].values)*100 / Baseline['Streaming Users'].values
    
    if AMZN_Streaming_Pct >= 0:
        AMZN_Streaming_Pct = round(AMZN_Streaming_Pct[0],1).astype(str) + '%'
    else:
        AMZN_Streaming_Pct = 'NA'
    
    # Spend
    if np.isnan(Q42021['spend'].values) or np.isnan(Baseline['spend'].values):
        Spend_Pct = 'NA'
    else:
        Spend_Pct =\
        (Q42021['spend'].values - Baseline['spend'].values)*100 / Baseline['spend'].values
        
        if Spend_Pct >= 0:
            Spend_Pct = round(Spend_Pct[0],1).astype(str) + '%'
    
        else:
            Spend_Pct = 'NA'
    
    result = pd.DataFrame({'Locale':Q42021['Locale'].values,'AMZN_Visitor_Pct':AMZN_Visitor_Pct,\
                          'AMZN_Streaming_Pct':AMZN_Streaming_Pct,'Spend_Pct':Spend_Pct})
    return result

In [ ]:
df_amm_grouped = df_amm.groupby('Locale', as_index=False).apply(AMM_Metric)

In [ ]:
df_all.dropna(inplace=True, axis=1, how='all')
df_all.dropna(inplace=True, axis=0)
df_all = df_all[df_all['vertical'] == 'QSR']
df_all.sort_values(['locale','slide'], inplace=True)
df_all['insights_num'] = df_all.groupby(['locale','slide'])['metric_value'].cumcount()+1

In [ ]:
df_all.head(20)

In [ ]:
# parameters

event = "Q4 Shopping Moments"
quarter = 'Q4'
vertical = 'QSR'
survey = 'Kantar'

# final number of locales is 13, not 14. Excluding CN
locales = ['US','CA','DE','IT','ES','UK','FR','JP','NL','AU','MX','UAE','BR']
audience = 'test'
#event_name_dict = {'US':'Q4 Shopping Moments','CA':'Q4 Shopping Moments','GB':'Q4 Shopping Moments',\
 #                  'DE':'Q4 Shopping Moments','IT':'Q4 Shopping Moments','ES':'Q4 Shopping Moments',\
  #                 'FR':'Q4 Shopping Moments','JP':'Q4 Shopping Moments','CN':'Q4 Shopping Moments',\
   #                'SE':'Q4 Shopping Moments','TR':'Q4 Shopping Moments','NL':'Sinterklaas','AU':'Boxing Day',\
    #               'MX':'Q4 Shopping Moments','UAE':'Q4 Shopping Moments','KSA':'Q4 Shopping Moments',\
     #              'BR':"Children’s Day"}

In [ ]:
ppt_template = 'Q4 NE Shopping Events Automated Playbook_QSR_Template.pptx'
#ppt = Presentation(ppt_template)

In [ ]:
def pptx_automation(locale):
    ppt = Presentation(ppt_template)
    # Automation

    # slide 3
    replace_text_in_slide(presentation=ppt,\
                      slidenumber=3, shape_name='Text Event', placeholder='[EVENT NAME]',\
                      new_value=event,font_size=20, alignment='LEFT',\
                      set_format=True, paragraph_spacing_ind=True)

    # slide 8
    replace_text_in_slide(presentation=ppt,\
                      slidenumber=8, shape_name='Text Event', placeholder='[EVENT NAME]',\
                      new_value=event, font_size=33)
    
    replace_text_in_slide_v2(presentation=ppt,\
                      slidenumber=8, shape_name='Text Event 2', placeholder='[EVENT NAME]',\
                      new_value=event,font_size=16, bold=True,\
                      symbol='', tag='',alignment = 'LEFT',\
                      font_red = 0, font_green=0, font_blue = 0, using_font_color=True,\
                      font_style='Amazon Ember Display')
        
    replace_text_in_slide_v2(presentation=ppt,\
                      slidenumber=8, shape_name='Text Event 3', placeholder='[EVENT NAME]',\
                      new_value=event,font_size=11, bold=False,\
                      symbol=' event goes live for millions of shoppers eager to shop', tag='',alignment = 'LEFT',\
                      font_red = 120, font_green=120, font_blue = 120, using_font_color=True,\
                      font_style='Amazon Ember Display')
    
    replace_text_in_notes(presentation=ppt, slidenumber=8, placeholder='[EVENT NAME]',\
                          new_value=event)
    
    # slide 9
    replace_text_in_slide(presentation=ppt,\
                      slidenumber=9, shape_name='Text Event', placeholder='[EVENT NAME]',\
                      new_value=event,font_size=90, set_format=True, alignment='LEFT')
    
    # slide 10
    replace_text_in_slide(presentation=ppt,\
                      slidenumber=10, shape_name='Text Event', placeholder='[EVENT NAME]',\
                      new_value=event,font_size=6.5)
    
    replace_text_in_slide(presentation=ppt,\
                      slidenumber=10, shape_name='Text Event', placeholder='{surveyed}',\
                      new_value=locale_size_dict['Final N'][locale],font_size=6.5)
    
    replace_text_in_slide_v2(presentation=ppt,\
                      slidenumber=10, shape_name='Text Insights 1', placeholder='{1}',\
                      new_value=extracting_insights(locale,'slide 10',1),font_size=36, bold=True,\
                      symbol='   ', tag='',alignment = 'CENTER',\
                      font_red = 255, font_green=255, font_blue = 255, using_font_color=True,\
                      font_style='Amazon Ember Display')
    
    replace_text_in_slide_v2(presentation=ppt,\
                      slidenumber=10, shape_name='Text Insights 2', placeholder='{1}',\
                      new_value=extracting_insights(locale,'slide 10',2),font_size=36, bold=True,\
                      symbol='   ', tag='',alignment = 'CENTER',\
                      font_red = 255, font_green=255, font_blue = 255, using_font_color=True,\
                      font_style='Amazon Ember Display')
    
    replace_text_in_slide_v2(presentation=ppt,\
                      slidenumber=10, shape_name='Text Insights 3', placeholder='{1}',\
                      new_value=extracting_insights(locale,'slide 10',3),font_size=36, bold=True,\
                      symbol='   ', tag='',alignment = 'CENTER',\
                      font_red = 255, font_green=255, font_blue = 255, using_font_color=True,\
                      font_style='Amazon Ember Display')
    
    replace_text_in_slide_v2(presentation=ppt,\
                      slidenumber=10, shape_name='Text Insights 4', placeholder='{1}',\
                      new_value=extracting_insights(locale,'slide 10',4),font_size=36, bold=True,\
                      symbol='   ', tag='',alignment = 'CENTER',\
                      font_red = 255, font_green=255, font_blue = 255, using_font_color=True,\
                      font_style='Amazon Ember Display')
    
    replace_text_in_notes(presentation=ppt, slidenumber=10, placeholder='[EVENT NAME]',\
                          new_value=event)
    
    # slide 11
    replace_text_in_notes(presentation=ppt, slidenumber=11, placeholder='[EVENT NAME]',\
                          new_value=event)
    
   # if locale in ['BR','NL','AU']:
    #      replace_text_in_slide(presentation=ppt,\
     #                 slidenumber=9, shape_name='Text Var1', placeholder='lead-up',\
      #                new_value='',font_size=90)

    # slide 12
    replace_text_in_slide(presentation=ppt,\
                      slidenumber=12, shape_name='Text Event', placeholder='[EVENT NAME]',\
                      new_value=event, font_size=90)


    # slide 13
    replace_text_in_slide(presentation=ppt,\
                      slidenumber=13, shape_name='Text Event', placeholder='[EVENT NAME]',\
                      new_value=event, font_size=33)
    
    replace_text_in_slide(presentation=ppt,\
                      slidenumber=13, shape_name='Text Event 2', placeholder='[EVENT NAME]',\
                      new_value=quarter, font_size=13)
    
    replace_text_in_slide(presentation=ppt,\
                      slidenumber=13, shape_name='Text Event 3', placeholder='[EVENT NAME]',\
                      new_value=quarter, font_size=13)
    
    
    replace_text_in_slide_v2(presentation=ppt,\
                      slidenumber=13, shape_name='Text Insights 1', placeholder='{1}',\
                      new_value=extracting_insights(locale,'slide 13',1),font_size=36, bold=True,\
                      symbol='', tag='',alignment = 'CENTER',\
                      font_red = 255, font_green=255, font_blue = 255, using_font_color=True,\
                      font_style='Amazon Ember Display')
    
    replace_text_in_slide_v2(presentation=ppt,\
                      slidenumber=13, shape_name='Text Insights 2', placeholder='{1}',\
                      new_value=extracting_insights(locale,'slide 13',2),font_size=36, bold=True,\
                      symbol='', tag='',alignment = 'CENTER',\
                      font_red = 255, font_green=255, font_blue = 255, using_font_color=True,\
                      font_style='Amazon Ember Display')
        
    replace_text_in_slide_v2(presentation=ppt,\
                      slidenumber=13, shape_name='Text Insights 3', placeholder='{1}',\
                      new_value=extracting_insights(locale,'slide 13',3),font_size=36, bold=True,\
                      symbol='', tag='',alignment = 'CENTER',\
                      font_red = 255, font_green=255, font_blue = 255, using_font_color=True,\
                      font_style='Amazon Ember Display')
    
    replace_text_in_slide(presentation=ppt,\
                      slidenumber=13, shape_name='Text Survey', placeholder='{surveyed}',\
                      new_value=locale_size_dict['Final N'][locale],font_size=6.5)
    
    replace_text_in_notes(presentation=ppt, slidenumber=13, placeholder='[EVENT NAME]',\
                          new_value=event)

    # slide 14
    replace_text_in_slide(presentation=ppt,\
                      slidenumber=14, shape_name='Text Event', placeholder='[EVENT NAME]',\
                      new_value=event, font_size=33)
    
    replace_text_in_slide(presentation=ppt,\
                      slidenumber=14, shape_name='Text Event 2', placeholder='[EVENT NAME]',\
                      new_value=quarter, font_size=13)
    
    replace_text_in_slide(presentation=ppt,\
                      slidenumber=14, shape_name='Text Event 3', placeholder='[EVENT NAME]',\
                      new_value=quarter, font_size=13)
    
    replace_text_in_slide(presentation=ppt,\
                      slidenumber=14, shape_name='Text Event 4', placeholder='[EVENT NAME]',\
                      new_value=quarter, font_size=13)
    
    replace_text_in_slide(presentation=ppt,\
                      slidenumber=14, shape_name='Text Locale', placeholder='[LOCALE]',\
                      new_value=locale + ' and Amazon Internal Data')
    
    # AMM insights
    replace_text_in_slide_v2(presentation=ppt,\
                      slidenumber=14, shape_name='Text Insights 1', placeholder='{1}',\
                      new_value=df_amm_grouped[df_amm_grouped['Locale']==locale]['AMZN_Visitor_Pct'].values[0],\
                      font_size=36, bold=True,\
                      symbol='', tag='',alignment = 'CENTER',\
                      font_red = 255, font_green=255, font_blue = 255, using_font_color=True,\
                      font_style='Amazon Ember Display')
    
    
    replace_text_in_slide_v2(presentation=ppt,\
                      slidenumber=14, shape_name='Text Insights 2', placeholder='{1}',\
                      new_value=df_amm_grouped[df_amm_grouped['Locale']==locale]['AMZN_Streaming_Pct'].values[0],\
                      font_size=36, bold=True,\
                      symbol='', tag='',alignment = 'CENTER',\
                      font_red = 255, font_green=255, font_blue = 255, using_font_color=True,\
                      font_style='Amazon Ember Display')
    
    replace_text_in_slide_v2(presentation=ppt,\
                      slidenumber=14, shape_name='Text Insights 3', placeholder='{1}',\
                      new_value=df_amm_grouped[df_amm_grouped['Locale']==locale]['Spend_Pct'].values[0],\
                      font_size=36, bold=True,\
                      symbol='', tag='',alignment = 'CENTER',\
                      font_red = 255, font_green=255, font_blue = 255, using_font_color=True,\
                      font_style='Amazon Ember Display')
    
    replace_text_in_notes(presentation=ppt, slidenumber=14, placeholder='[EVENT NAME]',\
                          new_value=event)
    
    
    # slide 15
    replace_text_in_slide_v2(presentation=ppt,\
                      slidenumber=15, shape_name='Text Insights 1', placeholder='{1}',\
                      new_value=extracting_insights(locale,'slide 15',1),font_size=36, bold=True,\
                      symbol='', tag='',alignment = 'CENTER',\
                      font_red = 255, font_green=255, font_blue = 255, using_font_color=True,\
                      font_style='Amazon Ember Display')
    
    replace_text_in_slide_v2(presentation=ppt,\
                      slidenumber=15, shape_name='Text Insights 2', placeholder='{1}',\
                      new_value=extracting_insights(locale,'slide 15',2),font_size=36, bold=True,\
                      symbol='', tag='',alignment = 'CENTER',\
                      font_red = 255, font_green=255, font_blue = 255, using_font_color=True,\
                      font_style='Amazon Ember Display')
        
    replace_text_in_slide_v2(presentation=ppt,\
                      slidenumber=15, shape_name='Text Insights 3', placeholder='{1}',\
                      new_value=extracting_insights(locale,'slide 15',3),font_size=36, bold=True,\
                      symbol='', tag='',alignment = 'CENTER',\
                      font_red = 255, font_green=255, font_blue = 255, using_font_color=True,\
                      font_style='Amazon Ember Display')
    
    replace_text_in_slide(presentation=ppt,\
                      slidenumber=15, shape_name='Text Event', placeholder='[EVENT NAME]',\
                      new_value=event,font_size=6.5)
    
    replace_text_in_slide(presentation=ppt,\
                      slidenumber=15, shape_name='Text Event', placeholder='{surveyed}',\
                      new_value=locale_size_dict['Final N'][locale],font_size=6.5)
    
    replace_text_in_notes(presentation=ppt, slidenumber=15, placeholder='[EVENT NAME]',\
                          new_value=event)

    # slide 17
    replace_text_in_slide(presentation=ppt,\
                      slidenumber=17, shape_name='Text Event', placeholder='[EVENT NAME]',\
                      new_value=event,font_size=33)

    replace_text_in_slide(presentation=ppt,\
                      slidenumber=17, shape_name='Text Event 2', placeholder='[EVENT NAME]',\
                      new_value=quarter, font_size=14)
    
    replace_text_in_slide(presentation=ppt,\
                      slidenumber=17, shape_name='Text Event 3', placeholder='[EVENT NAME]',\
                      new_value=quarter, font_size=14)
    
    replace_text_in_slide(presentation=ppt,\
                      slidenumber=17, shape_name='Text Event 4', placeholder='[EVENT NAME]',\
                      new_value=event, font_size=6.5)
    
    replace_text_in_slide(presentation=ppt,\
                      slidenumber=17, shape_name='Text Event 4', placeholder='{surveyed}',\
                      new_value=locale_size_dict['Final N'][locale],font_size=6.5)
    
            
    replace_text_in_slide_v2(presentation=ppt,\
                      slidenumber=17, shape_name='Text Insights 1', placeholder='{1}',\
                      new_value=extracting_insights(locale,'slide 17',1),font_size=36, bold=True,\
                      symbol='', tag='',alignment = 'CENTER',\
                      font_red = 255, font_green=255, font_blue = 255, using_font_color=True,\
                      font_style='Amazon Ember Display')
    
    replace_text_in_slide_v2(presentation=ppt,\
                      slidenumber=17, shape_name='Text Insights 2', placeholder='{1}',\
                      new_value=extracting_insights(locale,'slide 17',2),font_size=36, bold=True,\
                      symbol='', tag='',alignment = 'CENTER',\
                      font_red = 255, font_green=255, font_blue = 255, using_font_color=True,\
                      font_style='Amazon Ember Display')
    
    replace_text_in_notes(presentation=ppt, slidenumber=17, placeholder='[EVENT NAME]',\
                          new_value=event)
    
    # slide 18: remove lead-up for BR, NL and AU
    replace_text_in_slide(presentation=ppt,\
                      slidenumber=18, shape_name='Text Event', placeholder='[EVENT NAME]',\
                      new_value=event, font_size=90, set_format=True, alignment='LEFT')
    
    # slide 19
    replace_text_in_slide(presentation=ppt,\
                      slidenumber=19, shape_name='Text Event', placeholder='[EVENT NAME]',\
                      new_value=event,font_size=33)

    replace_text_in_slide(presentation=ppt,\
                      slidenumber=19, shape_name='Text Event 2', placeholder='[EVENT NAME]',\
                      new_value=quarter, font_size=14)
    
    replace_text_in_slide(presentation=ppt,\
                      slidenumber=19, shape_name='Text Event 3', placeholder='[EVENT NAME]',\
                      new_value=quarter, font_size=14)
    
    replace_text_in_slide(presentation=ppt,\
                      slidenumber=19, shape_name='Text Event 4', placeholder='[EVENT NAME]',\
                      new_value=event, font_size=6.5)
    
    replace_text_in_slide(presentation=ppt,\
                      slidenumber=19, shape_name='Text Event 4', placeholder='{surveyed}',\
                      new_value=locale_size_dict['Final N'][locale], font_size=6.5)
    
            
    replace_text_in_slide_v2(presentation=ppt,\
                      slidenumber=19, shape_name='Text Insights 1', placeholder='{1}',\
                      new_value=extracting_insights(locale,'slide 19',1),font_size=36, bold=True,\
                      symbol='', tag='',alignment = 'CENTER',\
                      font_red = 255, font_green=255, font_blue = 255, using_font_color=True,\
                      font_style='Amazon Ember Display')
    
    replace_text_in_slide_v2(presentation=ppt,\
                      slidenumber=19, shape_name='Text Insights 2', placeholder='{1}',\
                      new_value=extracting_insights(locale,'slide 19',2),font_size=36, bold=True,\
                      symbol='', tag='',alignment = 'CENTER',\
                      font_red = 255, font_green=255, font_blue = 255, using_font_color=True,\
                      font_style='Amazon Ember Display')

    replace_text_in_notes(presentation=ppt, slidenumber=19, placeholder='[EVENT NAME]',\
                          new_value=event)

    # slide 24
    replace_text_in_slide_v2(presentation=ppt,\
                      slidenumber=24, shape_name='Text Event', placeholder='[EVENT NAME]',\
                      new_value=event,font_size=14.7,font_style='Amazon Ember Display',\
                      bold=True, symbol='', tag='')
    
    replace_text_in_notes(presentation=ppt, slidenumber=24, placeholder='[EVENT NAME]',\
                          new_value=event)
    
    # slide 25
    replace_text_in_slide(presentation=ppt,\
                      slidenumber=25, shape_name='Text Event', placeholder='[EVENT NAME]',\
                      new_value=event,font_size=33)
    
    
    replace_text_in_slide_v2(presentation=ppt,\
                      slidenumber=25, shape_name='Text Event 2', placeholder='[EVENT NAME]',\
                      new_value='Total '+ event, font_size=20, bold=True,\
                      symbol='budget recommendation:', tag=' X',font_style='Amazon Ember Display',\
                      font_red = 255, font_green=255, font_blue = 255, using_font_color=True)
    # saving pptx file according to the naming convention:
    # NE_Q4_SHOPPING_MOMENTS_COUNTRYCODE_VERTICAL_AUDIENCE
    # example: NE_Q4_SHOPPING_MOMENTS_DE_Financial_Services_MasterCard
    if not os.path.exists('upload/'):
        os.mkdir('upload/')
    
    
    file = ppt.save('upload/NE_Q4_SHOPPING_MOMENTS_'+ locale +'_QSR.pptx')
    

def extracting_insights(locale, slide, insights_num):
    
    if any(df_all['locale']==locale):
        metric =\
        df_all[(df_all['locale']==locale) & (df_all['slide']==slide) &\
               (df_all['insights_num']==insights_num)]['metric_value'].values[0]
        
        metric = round(metric*100,1)

    else:
        metric = ''
    return str(metric) + '%'

In [ ]:
_ = [ pptx_automation(locale) for locale in locales]